### IMPORTS

In [1]:
import json
import time
import hashlib

In [2]:
from web3 import Web3

In [3]:
print('[X] Booting up...')

[X] Booting up...


### LOAD DATA FROM EXTERNAL FILES

In [4]:
with open('config/settings.json') as json_file:
    settings = json.load(json_file)

In [5]:
with open('config/identifier.json') as json_file:
    device_info = json.load(json_file)

In [6]:
with open('config/external.json') as json_file:
    external_info = json.load(json_file)

### HASH DEVICE IDENTIFIER

In [7]:
def hash_id(data):
    
    # CONVERT TO BYTES
    to_bytes = str.encode(str(data))
    
    # HASH ENCODED DATA
    hashed = hashlib.sha224(to_bytes).hexdigest()
    
    return hashed

In [8]:
device_id = hash_id(device_info)

### CONNECT TO CHAIN VIA WEBSOCKET

In [9]:
web3 = Web3(Web3.WebsocketProvider('ws://' + settings['gateways']['whisper']['host'] + ':' + str(settings['gateways']['whisper']['port'])))

In [10]:
if web3.isConnected():
    print('[X] Connected with gateway!')
else:
    print('[ ] Could not connect to gateway!')
    exit

[X] Connected with gateway!


### FETCH WHISPER API

In [11]:
shh = web3.geth.shh

### GENERATE WHISPER KEYPAIR

In [12]:
whisper_id = shh.newKeyPair()

In [13]:
print('[X] Generated keypair!')

[X] Generated keypair!


### SUBSCRIBE TO UPDATE EVENTS

In [14]:
print('[X] Listening for messages...\n')

[X] Listening for messages...



In [15]:
message_event = shh.newMessageFilter({
    'topic': web3.toHex(text=settings['whisper']['topic']['name']),
    'symKeyID': settings['whisper']['topic']['key']
})

In [ ]:
while(True):
    for event in shh.getMessages(message_event):
    
        # CONVERT MESSAGE PARAMS
        author = web3.toHex(event['sig'])
        nickname = author[0:4] + '...' + author[-4:]
        message = web3.toText(event['payload'])
        
        # CHECK THE FIRST WORD FOR TRIGGER
        first = message.split(' ')[0].lower()
        
        # IF A REQUEST IS FOUND
        if (first == '!request'):
            
            # SIGNAL REQUEST
            print('New request found!')
            
            # NUKE GARBAGE & SPLIT OUT INDIVIDUAL PARAMS
            checks = message[8:].replace(' ', '').split('&')
            
            # CHECK RESULTS
            results = []
            
            # LOOP THROUGH
            for pair in checks:
    
                # DECONSTRUCT KEY/VALUE
                key, value = pair.split('=')
            
                # MAKE SURE THE KEY EXISTS
                if (key in external_info):
            
                    # CHECK & PUSH PARAM COMPARISON RESULT
                    result = external_info[key] == value
                    results.append(result)
                
                # OTHERWISE, DEFAULT TO BAD RESULT
                else:
                    results.append(False)
                
            # IF ALL THE CHECKS MATCH
            if (results.count(False) == 0):
                
                # RESPONSE TEXT
                response = '@' + nickname + ' - I am available at ' + device_id
                
                # SLEEP FOR 2 SECONDS
                time.sleep(2)
                
                # RESPOND TO REQUEST
                shh.post({
                    'symKeyID': settings['whisper']['topic']['key'],
                    'payload': web3.toHex(text=response),
                    'topic': web3.toHex(text=settings['whisper']['topic']['name']),
                    'sig': whisper_id,
                    'powTarget': 2.5,
                    'powTime': 2
                })

New request found!
